# Predictions - Try #3


- Author: Elmo Chavez
- Date: 23-Aug-2023

**Description**

> [...]


## Read the Datasets


Libraries


In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
current_directory = os.getcwd()  # Get the current working directory
parent_directory = os.path.dirname(current_directory)  # Get the parent directory

In [3]:
import sys
sys.path.append(parent_directory)
import utils as feeg

In [4]:
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import optuna
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectPercentile, f_classif

import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb

Reading Datasets


In [5]:
filename_list = ['3_psd_bands_features_fp1.csv', '3_psd_bands_features_all.csv', 
                 '3_tfr_bands_features_fp1.csv', '3_tfr_bands_features_all.csv']

# Read all files
df_trainings = []
for file in filename_list:
    df_trainings.append(pd.read_csv(parent_directory+'/Training Datasets/'+file))

# Shape of the data
print('PSD Feature Extraction from Frequency Bands - FP1:',df_trainings[0].shape)
print('PSD Feature Extraction from Frequency Bands - All Channels:',df_trainings[1].shape)
print('TFR Feature Extraction from Frequency Bands - FP1:',df_trainings[2].shape)
print('TFR Feature Extraction from Frequency Bands - All Channels:',df_trainings[3].shape)

PSD Feature Extraction from Frequency Bands - FP1: (51, 45)
PSD Feature Extraction from Frequency Bands - All Channels: (51, 45)
TFR Feature Extraction from Frequency Bands - FP1: (51, 35)
TFR Feature Extraction from Frequency Bands - All Channels: (51, 35)


In [6]:
df_trainings[0].head()

,participant_id,Gender,Age,Group,MMSE,delta_total_power,delta_relative_power,delta_average_power,delta_spectral_entropy,delta_peak_to_peak,...,beta_kurtosis,beta_skewness,gamma_total_power,gamma_relative_power,gamma_average_power,gamma_spectral_entropy,gamma_peak_to_peak,gamma_std_dev,gamma_kurtosis,gamma_skewness
0,sub-002,1,78,0,22,2.384185e-09,0.768581,9.727398e-13,5.642890e-08,1.015567e-10,...,1.900457,1.475116,5.473149e-11,0.017644,2.233027e-14,1.574884e-09,1.740513e-12,9.280384e-14,8.284259,2.860125
1,sub-004,1,67,0,20,2.761436e-09,0.819641,1.126657e-12,6.488888e-08,1.342419e-10,...,4.535005,1.810307,1.225167e-10,0.036365,4.998641e-14,3.439100e-09,1.873167e-12,1.841240e-13,10.654699,3.272785
2,sub-005,0,70,0,22,2.843382e-09,0.798515,1.160090e-12,6.679794e-08,1.230672e-10,...,0.537760,0.935561,1.534990e-10,0.043108,6.262711e-14,4.268120e-09,2.029911e-12,2.379754e-13,0.293308,1.039158
3,sub-006,1,61,0,14,2.672633e-09,0.711554,1.090426e-12,6.290695e-08,1.211064e-10,...,-0.722982,0.472785,4.756680e-11,0.012664,1.940710e-14,1.381791e-09,4.853046e-13,6.950519e-14,2.242904,1.590105
4,sub-007,1,79,0,20,2.767352e-09,0.821226,1.129070e-12,6.513636e-08,1.168272e-10,...,6.826184,2.396031,4.902646e-11,0.014549,2.000264e-14,1.424083e-09,4.680290e-13,7.024027e-14,3.915154,1.531833


In [7]:
df_trainings[2].head()

,participant_id,Gender,Age,Group,MMSE,delta_total_power,delta_average_power,delta_peak_power,delta_std,delta_kurtosis,...,beta_peak_power,beta_std,beta_kurtosis,beta_skewness,gamma_total_power,gamma_average_power,gamma_peak_power,gamma_std,gamma_kurtosis,gamma_skewness
0,sub-002,1,78,0,22,0.019382,1.615161e-07,6.714592e-07,2.170376e-07,-0.321085,...,3.612499e-09,2.619153e-10,18.534499,2.747591,0.000095,4.531700e-11,3.106797e-09,9.963583e-11,282.069647,12.964888
1,sub-004,1,67,0,20,0.022172,1.847633e-07,7.405170e-07,2.480452e-07,-0.312793,...,3.277944e-09,1.794504e-10,5.031290,1.336889,0.000199,9.465428e-11,4.366202e-09,1.847765e-10,149.168250,8.822134
2,sub-005,0,70,0,22,0.022540,1.878299e-07,7.924840e-07,2.495321e-07,-0.321326,...,2.240047e-09,2.304361e-10,3.158683,1.444451,0.000236,1.125425e-10,2.232037e-09,1.920152e-10,10.463997,2.696169
3,sub-006,1,61,0,14,0.023753,1.979383e-07,8.168917e-07,2.736430e-07,-0.171644,...,7.074382e-09,1.431966e-09,3.228494,2.105543,0.000089,4.243826e-11,6.034054e-10,6.180919e-11,2.767625,1.847439
4,sub-007,1,79,0,20,0.020673,1.722733e-07,7.221850e-07,2.245220e-07,-0.316826,...,1.580119e-09,2.337696e-10,1.041545,1.289885,0.000088,4.194346e-11,4.375251e-10,6.052442e-11,2.135301,1.736876


## Predictions


In [8]:
df_results = pd.DataFrame()

def get_X_y_split(df, cols_to_remove, target):
    feature_names = df.drop(columns=cols_to_drop, axis=1).columns.tolist()
    X = df.drop(columns=cols_to_drop, axis=1).values
    y = df[target].values
    
    return X, y, feature_names

### PSD Features


#### FP1


In [10]:
# Variables for Results
approach = 'Frequency Domain (PSD)'
frequencies = 'Delta, Theta, Alpha, Beta, Gamma'
channels = 'FP1'

cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

# Training data
X, y, features = get_X_y_split(df_trainings[0], cols_to_drop, target)

# Predictions
psd_fp1_predictions = feeg.predictor(X, y, approach, frequencies, channels, features, 15, 35)
df_results_psd_fp1 = pd.DataFrame(psd_fp1_predictions)
df_results_psd_fp1.sort_values('Accuracy', ascending=False)

Running SVM
Running Random Forest
Running AdaBoost
Running LightGBM


,Feature Extraction,Frequencies,Channels,Metrics,Cross-Validate Method,Feature Selection,Metrics Selected,ML Model,Accuracy
22,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[delta_total_power, delta_relative_power, delt...",Random Forest,0.709091
20,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_total_power, delta_relative_power, delt...",Random Forest,0.672727
34,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[delta_total_power, delta_relative_power, delt...",AdaBoost,0.672727
32,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_total_power, delta_relative_power, delt...",AdaBoost,0.672727
37,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectKBest - mutual_info_classif,"[delta_relative_power, delta_std_dev, theta_re...",LightGBM,0.665455
3,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectPercentile - mutual_info_classif,"[delta_relative_power, delta_std_dev, theta_re...",SVM,0.645455
47,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Age, delta_peak_to_peak, delta_skewness, thet...",LightGBM,0.636364
46,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[delta_total_power, delta_relative_power, delt...",LightGBM,0.636364
45,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[Age, delta_peak_to_peak, delta_skewness, thet...",LightGBM,0.636364
44,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_total_power, delta_relative_power, delt...",LightGBM,0.636364


#### All Channels


In [11]:
# Variables for Results
approach = 'Frequency Domain (PSD)'
frequencies = 'Delta, Theta, Alpha, Beta, Gamma'
channels = 'All'

cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

# Training data
X, y, features = get_X_y_split(df_trainings[1], cols_to_drop, target)

# Predictions
psd_all_predictions = feeg.predictor(X, y, approach, frequencies, channels, features, 15, 35)
df_results_psd_all = pd.DataFrame(psd_all_predictions)
df_results_psd_all.sort_values('Accuracy', ascending=False)

Running SVM
Running Random Forest
Running AdaBoost
Running LightGBM


,Feature Extraction,Frequencies,Channels,Metrics,Cross-Validate Method,Feature Selection,Metrics Selected,ML Model,Accuracy
38,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectPercentile - f_classif,"[Gender, delta_total_power, delta_relative_pow...",LightGBM,0.705455
36,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectKBest - f_classif,"[Gender, delta_total_power, delta_relative_pow...",LightGBM,0.705455
4,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedKFold,SelectKBest - f_classif,"[Gender, Age, delta_total_power, delta_relativ...",SVM,0.647273
6,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedKFold,SelectPercentile - f_classif,"[Gender, Age, delta_total_power, delta_relativ...",SVM,0.647273
0,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectKBest - f_classif,"[Gender, delta_total_power, delta_relative_pow...",SVM,0.645455
2,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectPercentile - f_classif,"[Gender, delta_total_power, delta_relative_pow...",SVM,0.645455
3,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectPercentile - mutual_info_classif,"[delta_total_power, delta_relative_power, delt...",SVM,0.645455
11,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Age, delta_total_power, delta_average_power, ...",SVM,0.636364
46,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Gender, Age, delta_total_power, delta_relativ...",LightGBM,0.636364
45,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[Age, delta_total_power, delta_average_power, ...",LightGBM,0.636364


### TFR Features


#### FP1


In [12]:
# Variables for Results
approach = 'Time-Frequency Domain (TFR)'
frequencies = 'Delta, Theta, Alpha, Beta, Gamma'
channels = 'FP1'

cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

# Training data
X, y, features = get_X_y_split(df_trainings[2], cols_to_drop, target)

# Predictions
tfr_fp1_predictions = feeg.predictor(X, y, approach, frequencies, channels, features, 15, 35)
df_results_tfr_fp1 = pd.DataFrame(tfr_fp1_predictions)
df_results_tfr_fp1.sort_values('Accuracy', ascending=False)

Running SVM
Running Random Forest
Running AdaBoost
Running LightGBM


,Feature Extraction,Frequencies,Channels,Metrics,Cross-Validate Method,Feature Selection,Metrics Selected,ML Model,Accuracy
34,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Age, theta_average_power, theta_std, beta_tot...",AdaBoost,0.690909
22,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Age, theta_average_power, theta_std, beta_tot...",Random Forest,0.690909
20,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedShuffleSplit,SelectKBest - f_classif,"[Gender, Age, theta_total_power, theta_average...",Random Forest,0.654545
32,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedShuffleSplit,SelectKBest - f_classif,"[Gender, Age, theta_total_power, theta_average...",AdaBoost,0.654545
18,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectPercentile - f_classif,"[Age, theta_peak_power, theta_std, beta_total_...",Random Forest,0.649091
16,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectKBest - f_classif,"[Gender, Age, delta_skewness, theta_total_powe...",Random Forest,0.647273
4,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectKBest - f_classif,"[Gender, Age, delta_skewness, theta_total_powe...",SVM,0.647273
5,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Age, delta_total_power, delta_average_power, ...",SVM,0.647273
6,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectPercentile - f_classif,"[Age, theta_peak_power, theta_std, beta_total_...",SVM,0.647273
7,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[delta_total_power, delta_peak_power, delta_st...",SVM,0.647273


#### All Channels


In [13]:
# Variables for Results
approach = 'Time-Frequency Domain (TFR)'
frequencies = 'Delta, Theta, Alpha, Beta, Gamma'
channels = 'All'

cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

# Training data
X, y, features = get_X_y_split(df_trainings[3], cols_to_drop, target)

# Predictions
tfr_all_predictions = feeg.predictor(X, y, approach, frequencies, channels, features, 15, 35)
df_results_tfr_all = pd.DataFrame(tfr_all_predictions)
df_results_tfr_all.sort_values('Accuracy', ascending=False)

Running SVM
Running Random Forest
Running AdaBoost
Running LightGBM


,Feature Extraction,Frequencies,Channels,Metrics,Cross-Validate Method,Feature Selection,Metrics Selected,ML Model,Accuracy
36,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectKBest - f_classif,"[Gender, delta_total_power, delta_average_powe...",LightGBM,0.765455
38,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectPercentile - f_classif,"[delta_total_power, delta_average_power, delta...",LightGBM,0.685455
28,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectKBest - f_classif,"[Gender, Age, delta_total_power, delta_average...",AdaBoost,0.670909
14,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectPercentile - f_classif,"[delta_total_power, delta_average_power, delta...",Random Forest,0.649091
4,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectKBest - f_classif,"[Gender, Age, delta_total_power, delta_average...",SVM,0.647273
5,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Age, delta_total_power, delta_average_power, ...",SVM,0.647273
6,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectPercentile - f_classif,"[Age, delta_total_power, delta_average_power, ...",SVM,0.647273
7,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Gender, delta_total_power, delta_average_powe...",SVM,0.647273
0,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectKBest - f_classif,"[Gender, delta_total_power, delta_average_powe...",SVM,0.645455
39,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectPercentile - mutual_info_classif,"[delta_total_power, delta_average_power, theta...",LightGBM,0.645455


Save Results


In [14]:
df_results = pd.concat([df_results_psd_fp1, df_results_psd_all, df_results_tfr_fp1, df_results_tfr_all], ignore_index=True)
df_results.sort_values('Accuracy', ascending=False).head(20)

,Feature Extraction,Frequencies,Channels,Metrics,Cross-Validate Method,Feature Selection,Metrics Selected,ML Model,Accuracy
180,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectKBest - f_classif,"[Gender, delta_total_power, delta_average_powe...",LightGBM,0.765455
22,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[delta_total_power, delta_relative_power, delt...",Random Forest,0.709091
86,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectPercentile - f_classif,"[Gender, delta_total_power, delta_relative_pow...",LightGBM,0.705455
84,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_relativ...",KFold,SelectKBest - f_classif,"[Gender, delta_total_power, delta_relative_pow...",LightGBM,0.705455
118,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Age, theta_average_power, theta_std, beta_tot...",Random Forest,0.690909
130,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_average...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Age, theta_average_power, theta_std, beta_tot...",AdaBoost,0.690909
182,Time-Frequency Domain (TFR),"Delta, Theta, Alpha, Beta, Gamma",All,"[Gender, Age, delta_total_power, delta_average...",KFold,SelectPercentile - f_classif,"[delta_total_power, delta_average_power, delta...",LightGBM,0.685455
20,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_total_power, delta_relative_power, delt...",Random Forest,0.672727
34,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[delta_total_power, delta_relative_power, delt...",AdaBoost,0.672727
32,Frequency Domain (PSD),"Delta, Theta, Alpha, Beta, Gamma",FP1,"[Gender, Age, delta_total_power, delta_relativ...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_total_power, delta_relative_power, delt...",AdaBoost,0.672727


In [15]:
filename = '3_results.csv'
df_results.to_csv(current_directory+'/'+filename)

## Hyperparameters Optimization


In [16]:
def get_Split_Xy_Best_Features(X, y, test_size=0.2, selector_k=10, train_test=True):
    
    selector = SelectKBest(score_func=f_classif, k=selector_k)
    
    if train_test:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)
        #selected_index = selector.get_support()
        
        return X_train_selected, X_test_selected, y_train, y_test
    else:
        X_selected = selector.fit_transform(X, y)
        return X_selected


Selecting the model with the best performance


**TFR All Channels**


In [17]:
# Train Test Split
cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

X, y, features = get_X_y_split(df_trainings[3], cols_to_drop, target)
X_train, X_test, y_train, y_test = get_Split_Xy_Best_Features(X, y, 0.2, 15, True)

**Grid_Search**


In [18]:
# Define the parameter grid for grid search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_child_samples': [5, 10, 20, 30]
    #'colsample_bytree': [0.7, 0.9, 1]
}

# model
lightgbm_classifier = lgb.LGBMClassifier()

# GridSearchCV
grid_search = GridSearchCV(estimator=lightgbm_classifier, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(best_params)

# Predict with the best model
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

{'learning_rate': 0.01, 'max_depth': 3, 'min_child_samples': 20, 'n_estimators': 50}
Accuracy: 0.7272727272727273


**Function Optuna**


In [19]:
# Function for Optuna
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0)
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Initialize Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters and best accuracy
best_params = study.best_params
best_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

[I 2023-08-27 12:27:04,289] A new study created in memory with name: no-name-e77b7b72-258f-4ed6-a033-0c211a183aec
[I 2023-08-27 12:27:04,302] Trial 0 finished with value: 0.7272727272727273 and parameters: {'learning_rate': 0.16011882098027283, 'n_estimators': 346, 'max_depth': 6, 'min_child_samples': 32, 'colsample_bytree': 0.5262371894411431}. Best is trial 0 with value: 0.7272727272727273.
[I 2023-08-27 12:27:04,315] Trial 1 finished with value: 0.7272727272727273 and parameters: {'learning_rate': 0.12884415383593878, 'n_estimators': 406, 'max_depth': 10, 'min_child_samples': 44, 'colsample_bytree': 0.7344560469214272}. Best is trial 0 with value: 0.7272727272727273.
[I 2023-08-27 12:27:04,321] Trial 2 finished with value: 0.7272727272727273 and parameters: {'learning_rate': 0.21117382901974044, 'n_estimators': 397, 'max_depth': 4, 'min_child_samples': 20, 'colsample_bytree': 0.5252978989426276}. Best is trial 0 with value: 0.7272727272727273.
[I 2023-08-27 12:27:04,336] Trial 3 fin

Best Hyperparameters: {'learning_rate': 0.15198854472148676, 'n_estimators': 431, 'max_depth': 4, 'min_child_samples': 19, 'colsample_bytree': 0.5748989518721298}
Best Accuracy: 0.8181818181818182


Compare with **TFR Channel FP1**


In [20]:
# Train Test Split
cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

X, y, features = get_X_y_split(df_trainings[2], cols_to_drop, target)
X_train, X_test, y_train, y_test = get_Split_Xy_Best_Features(X, y, 0.2, 15, True)

In [21]:
# Function for Optuna
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0)
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Initialize Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters and best accuracy
best_params = study.best_params
best_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

[I 2023-08-27 12:27:11,245] A new study created in memory with name: no-name-2004cb8b-c519-4ba0-9f4a-2f091ce6b7f9
[I 2023-08-27 12:27:11,258] Trial 0 finished with value: 0.7272727272727273 and parameters: {'learning_rate': 0.08273115592351894, 'n_estimators': 262, 'max_depth': 3, 'min_child_samples': 22, 'colsample_bytree': 0.6128788382645466}. Best is trial 0 with value: 0.7272727272727273.
[I 2023-08-27 12:27:11,267] Trial 1 finished with value: 0.7272727272727273 and parameters: {'learning_rate': 0.18150463797866334, 'n_estimators': 329, 'max_depth': 5, 'min_child_samples': 44, 'colsample_bytree': 0.5235857027157703}. Best is trial 0 with value: 0.7272727272727273.
[I 2023-08-27 12:27:11,276] Trial 2 finished with value: 0.7272727272727273 and parameters: {'learning_rate': 0.2849439619173208, 'n_estimators': 195, 'max_depth': 10, 'min_child_samples': 26, 'colsample_bytree': 0.698881126697267}. Best is trial 0 with value: 0.7272727272727273.
[I 2023-08-27 12:27:11,280] Trial 3 finis

Best Hyperparameters: {'learning_rate': 0.08273115592351894, 'n_estimators': 262, 'max_depth': 3, 'min_child_samples': 22, 'colsample_bytree': 0.6128788382645466}
Best Accuracy: 0.7272727272727273


Selecting the model with the second best performance


**PSD FP1 Channel**


In [22]:
# Train Test Split
cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

X, y, features = get_X_y_split(df_trainings[0], cols_to_drop, target)
#X_train, X_test, y_train, y_test = get_Split_Xy_Best_Features(X, y, 0.2, 15, true)
X_selected = get_Split_Xy_Best_Features(X, y, 0.2, 15, False)

In [26]:
# Define the objective function
def objective(params):
    model = RandomForestClassifier(**params)
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Define the search space
param_grid = {
    'n_estimators': [50,100,150,200,250,300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Set up the optimization study
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')

# Run the optimization
grid_search.fit(X_selected, y)

# Print the best hyperparameters and the best accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best Accuracy: 0.6890909090909092


**PSD All Channels**


In [27]:
# Train Test Split
cols_to_drop = ['participant_id','Group','MMSE']
target = 'Group'

X, y, features = get_X_y_split(df_trainings[1], cols_to_drop, target)
#X_train, X_test, y_train, y_test = get_Split_Xy_Best_Features(X, y, 0.2, 15)
X_selected = get_Split_Xy_Best_Features(X, y, 0.2, 15, False)

In [29]:
# Define the objective function
def objective(params):
    model = RandomForestClassifier(**params)
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Define the search space
param_grid = {
    'n_estimators': [50,100,150,200,250,300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Set up the optimization study
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')

# Run the optimization
grid_search.fit(X_selected, y)

# Print the best hyperparameters and the best accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 50}
Best Accuracy: 0.7836363636363636
